# Aufgabe 2: Hoch- und Tiefpassfilter
Hochpass- und Tiefpassfilter im Frequenzbereich erlauben Kantenextraktion bzw. Glättung von Bildern, analog zu Faltungen im Ortsbereich.
Dazu werden im fouriertransformierten Bild alle Fourierkoeffizienten innerhalb (idealer Hochpassfilter) bzw. außerhalb (idealer Tiefpassfilter) eines Radius $D_0$ um den Bildmittelpunkt auf $0$ gesetzt.

Verwenden Sie die von `numpy.fft` bereitgestellte Funktionalität zur Berechnung der Fouriertransformation eines Bildes!
Wenden Sie im Frequenzraum jeweils einen idealen Hochpass- und einen idealen Tiefpassfilter an!
Transformieren Sie das veränderte Spektrum zurück in den Ortsbereich (`ifftshift`, `ifft2`) und interpretieren Sie die Ergebnisse!

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import imageio
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition des Filters

Definieren Sie den Filter als binärwertige Funktion in Abhängigkeit vom Radius $D$ und der Konstante $D_0$.

In [ ]:
D0 = 20
lowpass_filter = lambda D: int(D<=D0)
highpass_filter = lambda D: int(D>D0)

## 2. Laden und Normalisieren des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

In [ ]:
image = image.astype(np.float32)
image -= image.min()
image /= image.max()

## 3. Berechnung der Fouriertransformation
Berechnen Sie nun die Fouriertransformation von `image`, indem Sie die entsprechende Funktion aus `numpy.fft` verwenden. Achten Sie darauf, das Spektrum zu zentrieren!

In [ ]:
image_transformed = np.fft.fftshift(np.fft.fft2(image))

## 4. Filterung

Definieren Sie nun eine Funktion `ex3_filter_spectrum`, die einen gegebenen radiusabhängigen Filter auf ein bereits Fouriertransformiertes Bild anwendet:

In [ ]:
def ex3_filter_spectrum(spectrum, radial_filter):
    copy_spectrum = spectrum.copy()
    y, x = image.shape

    for v in range(y):
        for u in range(x):
            D = np.sqrt((v-y/2)**2 + (u-x/2)**2)
            copy_spectrum[v,u] *= radial_filter(D)

    return copy_spectrum

Das transformierte Bild (Spektrum) wird nun gefiltert:

In [ ]:
image_transformed_lowpass = ex3_filter_spectrum(image_transformed, lowpass_filter)
image_transformed_highpass = ex3_filter_spectrum(image_transformed, highpass_filter)

## 5. Inverse Filterung
Das veränderte Spektrum soll nun in den Ortsbereich zurücktransformiert werden. Verwenden Sie dazu die entsprechenden Funktionen des Paketes `numpy.fft`.

In [ ]:
image_filter_lowpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_lowpass)).real
image_filter_highpass = np.fft.ifft2(np.fft.ifftshift(image_transformed_highpass)).real

Vergleichen Sie nun das gefilterte Bild mit dem Originalbild:

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1,3,1, title='Original Image')
plt.imshow(image, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,3,2, title='Ideal Lowpass')
plt.imshow(image_filter_lowpass, cmap='gray', vmin=0, vmax=1)
plt.subplot(1,3,3, title='Ideal Highpass')
plt.imshow(image_filter_highpass, cmap='gray')
plt.show()